In [1]:
import pandas as pd
from shapely.geometry import Point, mapping
from fiona import collection

inCSV  = 'C:/lidar/2012GIS6345.csv'
shpOut = '2012.shp'
lon = 'lon'
lat = 'lat'

schema = { 'geometry': 'Point', 'properties': { 'name': 'str' } }

df = pd.read_csv(inCSV) 

data = df
with collection(shpOut, "w", "ESRI Shapefile", schema) as output:
    for index, row in data.iterrows():
        point = Point(row[lon], row[lat])
        output.write({
        'properties': {'name': row['name']},
        'geometry': mapping(point)
})

In [2]:
#creating shapefile from .csv file

In [3]:
#this code made 2012 shapefile

In [4]:
import pandas as pd
from shapely.geometry import Point, mapping
from fiona import collection

inCSV  = 'C:/lidar/moose_harvest_2009.csv'
shpOut = '2009.shp'
lon = 'lon'
lat = 'lat'

schema = { 'geometry': 'Point', 'properties': { 'name': 'str' } }

df = pd.read_csv(inCSV) 

data = df
with collection(shpOut, "w", "ESRI Shapefile", schema) as output:
    for index, row in data.iterrows():
        point = Point(row[lon], row[lat])
        output.write({
        'properties': {'name': row['name']},
        'geometry': mapping(point)
})
        
#created a shapefile for 2009 too
#plan is to combine them together into one shapefile through geopandas

In [5]:
import pandas as pd
import geopandas as gpd

gdf1 = gpd.read_file('2012.shp')
gdf2 = gpd.read_file('2009.shp')

gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2]))

In [6]:
#creating a buffer around the 2009 data for Moose harvest

In [7]:
from shapely.geometry import mapping, shape

from fiona import collection



with collection("2009.shp", "r") as input:

    # schema = input.schema.copy()

    schema = { 'geometry': 'Polygon', 'properties': { 'name': 'str' } }

    with collection(

        "2009_realbuffer.shp", "w", "ESRI Shapefile", schema) as output:

        for point in input:

            output.write({

                'properties': {

                    'name': point['properties']['name']

                },

                'geometry': mapping(shape(point['geometry']).buffer(5.0))

            })

In [8]:
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
from fiona import collection

with collection("2009_realbuffer.shp", "r") as input:
    schema = input.schema.copy()
    with collection(
            "2009_realunion.shp", "w", "ESRI Shapefile", schema) as output:
        shapes = []
        for f in input:
            shapes.append(shape(f['geometry']))
        merged = cascaded_union(shapes)
        output.write({
            'properties': {
                'name': 'Buffer Area'
                },
            'geometry': mapping(merged)
            })